# Connecting Social Media via Domain Names
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import psycopg2
import glob
import json
import pathlib
import os
import subprocess
import pandas as pd
from sql_queries import reddit_comments_table_create, git_metadata_table_create, list_table_names, reddit_comments_table_insert
from modules.gitindexer import index_git_repos, get_repo_metadata_remote, get_repo_metadata_path
import modules.parsers as parsers
import configparser

In [2]:
# We will use postgres
config = configparser.ConfigParser()
config.sections()
config.read('dl.cfg')
DB_USER=    config['POSTGRES']['DB_USER']
DB_PASSWORD=config['POSTGRES']['DB_PASSWORD']
DB_HOST=    config['POSTGRES']['DB_HOST']
DB_PORT=    config['POSTGRES']['DB_PORT']
DB_NAME=    config['POSTGRES']['DB_NAME']
psql_conn_string="postgresql://{}:{}@{}:{}/{}".format(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT,DB_NAME)
# print("psql " + psql_conn_string)

In [ ]:
conn = psycopg2.connect(psql_conn_string)
cur = conn.cursor()

### Step 1: Scope the Project and Gather Data

#### Scope 

The purpose of this project is to develop a ETL pipeline for social media information, in this case reddit comments and git metadata. The unique quality across these datasets are domain names. Git metadata includes email addresses which use a domain name and website URL's can be extracted from reddit comments.

#### Describe and Gather Data 

Two datasets were used for this project,

* Reddit comment export from [pushift.io](https://files.pushshift.io/reddit/comments/daily/), specically the RC_2018-01-01 export
* A custom list of git repos with metadata extracted using a custom script

* Cross dataset comparisons
  * Join domain name of git email with domain name of URL in reddit comment
  * Join username from git email with reddit username
* Inner dataset queries
  * Reddit
    * Most comments per user
    * Most comments per subreddit
  * Git
    * Groupby Email Address
    * Group by email address AND repo

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

**Reddit Comment**

The main problem with the reddit dataset is that it is too large with various files containing JSON files 1-5 Gb in size. I split these files up to make ingestion easier.

**Git data**

I surveyed a lot of different tools to extract metadata from git repos. I discovered [mergestat](https://github.com/mergestat/mergestat) a simple open source program that allows one to perform SQL queries on all data in a git repository and even includes a ndjson export therefore the data can be ingested strait into pandas without a second thought.

#### Cleaning Steps

**Reddit Comment**

I had to create a schema for the reddit comments and I did not have a use for some of the key value pairs. I did not want to import all the key value pairs within the export therefore I selected a subset of columns from the dataframe.

**Git data**

There was a problem with mergestat though. There was no way to get the remote_url of the git repo while querying SQL. To solve this I extracted the remote URL myself and fed in a hard coded column into the SQL query. To see how I solved this check out modules/gitindexer.py

In [4]:
# Performing cleaning tasks here
# Split large ndjson file with reddit comments

p = pathlib.Path("./data/reddit_split")
p.mkdir(parents=True, exist_ok=True)
reddit_split_command = "cd data && split -l 1000 RC_2018-01-01 reddit_split/reddit_2018_01_01_"
process = subprocess.Popen(reddit_split_command, shell=True)

In [5]:
# Load reddit data into postgres

parsers.drop_tables(psql_conn_string, list_table_names)
parsers.drop_tables(psql_conn_string, ["reddit_comments2", "test_reddit_comments"])
sub_df = ["author", "body", "score", "can_gild", "controversiality", "created_utc", "edited", "id", "is_submitter", "link_id", "parent_id", "permalink", "subreddit"]
parsers.json_glob_to_database("./data/reddit_split/**", psql_conn_string, "reddit_comments", sub_df)

# TESTING
# parsers.json_glob_to_database("./data/reddit_split/reddit_2018_01_01_bf", psql_conn_string, "test_reddit_comments", sub_df)

In [6]:
# Clone git repos

p = pathlib.Path("./data/cloned-repos")
p.mkdir(parents=True, exist_ok=True)
git_repo_urls = json.load(open("./data/git_repos.json"))
for repo_url in git_repo_urls:
    git_clone_command = "cd ./data/cloned-repos && git clone {}".format(repo_url)
    # os.system(reddit_split_command) # subprocess does not always work

In [7]:
# Get git repo metadata
git_paths = glob.glob("./data/cloned-repos/**",)
for git_path in git_paths:
    # print("Indexing", git_path)
    try:
        pass
        # get_repo_metadata_path(git_path, "commits", "./data/git_out/")
    except:
        print("Got an error")

In [8]:
# Load git_metadata into postgres

parsers.drop_tables(psql_conn_string, ["git_metadata", "git_metadata2", "test_git_metadata"])
parsers.json_glob_to_database("./data/git_out/*/*/*.json", psql_conn_string, "git_metadata")

# TESTING
# parsers.json_glob_to_database("./data/git_out/commits/livepeer/livepeerjs.json", psql_conn_string, "test_git_metadata")

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

We have two fact tables. One fact table is git_metadata, number of commits, email, remote_url. The other fact table is reddit_comments which contains the author, subreddit, time of submisson etc. etc.

#### 3.2 Mapping Out Data Pipelines

There are two separate data pipelines, one for each fact table. Then dimmension tables can be created to store joins, aggregation and other results.

Rather than store the domain name in a separate table I wrote a function that adds a column and performs a regex query in order to extract the data I desired.

**git_metadata fact talbe**

1. Load data into database
  * For this I just read all the processed JSON files using glob then load each one of them into the SQL database
2. Add email_username column, update column with username from author_email column using regex
3. Add email_domain column, update column with username from author_email column using regex

**reddit_comments fact table**

1. Load data into database
2. Add url column, update column with url from body column using regex
3. Add domainname column, update column with domainname from url column  using regex

**shared_domains dimmension table**

## TODO

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [9]:
# Write code here

# parsers.sql_extract_url(psql_conn_string, "test_reddit_comments", "body", "url")
# parsers.sql_parse_email_domainname(psql_conn_string, "test_git_metadata", "author_email", "email_domain")
# parsers.sql_extract_domain_from_url_regex(psql_conn_string, "test_reddit_comments", "url", "domainname")
# parsers.sql_parse_email_username(psql_conn_string, "test_git_metadata", "author_email", "email_username")

In [11]:
parsers.sql_extract_url(psql_conn_string, "reddit_comments", "body", "url")

In [14]:
parsers.sql_parse_email_domainname(psql_conn_string, "git_metadata", "author_email", "email_domain")

In [15]:
parsers.sql_parse_email_username(psql_conn_string, "git_metadata", "author_email", "email_username")

In [16]:
parsers.sql_extract_domain_from_url_regex(psql_conn_string, "reddit_comments", "url", "domainname")

In [17]:
# Number of joined domain names
number_domain_name_query = """
SELECT COUNT((reddit_comments.domainname))
    FROM reddit_comments JOIN git_metadata 
    ON reddit_comments.domainname = git_metadata.email_domain;
"""
number_domain_name_result = pd.read_sql_query(number_domain_name_query ,psql_conn_string)


In [19]:
# List of joined domain names
list_joined_domain_names_query = """
SELECT DISTINCT(reddit_comments.domainname) 
    FROM reddit_comments JOIN git_metadata 
    ON reddit_comments.domainname = git_metadata.email_domain;
"""
joined_domain_names_result = pd.read_sql_query(list_joined_domain_names_query ,psql_conn_string)

In [24]:
# List of joined usernames
count_joined_user_names_query = """
SELECT COUNT(DISTINCT(reddit_comments.author))
    FROM reddit_comments JOIN git_metadata 
    ON reddit_comments.author = git_metadata.email_username;
"""
count_usernames_result = pd.read_sql_query(count_joined_user_names_query ,psql_conn_string)

In [30]:
print("\nNumber of results joining the datasets by domain name")
print(number_domain_name_result)
print("\nList of overlapping domain names")
print(joined_domain_names_result)
print("\nNumber of emails that have the same username as a reddit user")
print(count_usernames_result)
print("Getting no resulsts here is interesting, increasing the dataset should eventually lead to a collusion")


Number of results joining the datasets by domain name
   count
0  17508

List of overlapping domain names
              domainname
0           {amazon.com}
1               {ark.io}
2            {augur.net}
3          {binance.com}
4            {bitso.com}
5         {dogecoin.com}
6               {dss.co}
7   {freenetproject.org}
8        {getmonero.org}
9           {github.com}
10          {golang.org}
11          {google.com}
12           {hackmd.io}
13       {instagram.com}
14             {iohk.io}
15            {iota.org}
16   {lightning.network}
17        {loopring.org}
18         {mailbox.org}
19       {mainframe.com}
20        {makerdao.com}
21             {neo.org}
22        {nextdoor.com}
23       {pinterest.com}
24           {prezi.com}
25            {purse.io}
26      {radarrelay.com}
27            {sia.tech}
28        {snapchat.com}
29      {soundcloud.com}
30         {steemit.com}
31       {travis-ci.org}
32          {trello.com}
33         {unity3d.com}
34        {uwaterl

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [32]:
# Check total counts on all tables
tmp_query = "SELECT COUNT(*) FROM reddit_comments;"
num_reddit_comments = pd.read_sql_query(tmp_query ,psql_conn_string)
tmp_query = "SELECT COUNT(*) FROM git_metadata;"
num_git_metadata = pd.read_sql_query(tmp_query ,psql_conn_string)

In [62]:
# Check distinct primary key count on each table

tmp_query = "SELECT COUNT(DISTINCT(p_key)) FROM reddit_comments;"
num_reddit_comments_primary_key = pd.read_sql_query(tmp_query ,psql_conn_string)
tmp_query = "SELECT COUNT(DISTINCT(p_key)) FROM git_metadata;"
num_git_metadata_primary_key = pd.read_sql_query(tmp_query ,psql_conn_string)

In [38]:
# Check distinct domain_names on all tables
tmp_query = "SELECT DISTINCT(domainname) FROM reddit_comments;"
num_reddit_comments_domain_names = pd.read_sql_query(tmp_query ,psql_conn_string)
tmp_query = "SELECT DISTINCT(email_domain) FROM git_metadata;"
num_git_metadata_domain_names = pd.read_sql_query(tmp_query ,psql_conn_string)

In [40]:
# Check distinct users on all tables
tmp_query = "SELECT DISTINCT(author) FROM reddit_comments;"
num_reddit_comments_primary_key = pd.read_sql_query(tmp_query ,psql_conn_string)
tmp_query = "SELECT DISTINCT(email_username) FROM git_metadata;"
num_reddit_comments_primary_key = pd.read_sql_query(tmp_query ,psql_conn_string)

In [66]:
if num_reddit_comments[["count"]].values[0][0] == num_reddit_comments_primary_key[["count"]].values[0][0]:
    print("Number of primary keys for reddit_comments matches number of rows")
else:
    print("We got a problem")

Number of primary keys for reddit_comments matches number of rows


In [67]:
if num_git_metadata[["count"]].values[0][0] == num_git_metadata_primary_key[["count"]].values[0][0]:
    print("Number of primary keys for reddit_comments matches number of rows")
else:
    print("We got a problem")

Number of primary keys for reddit_comments matches number of rows


In [72]:
print("There are", str(len(num_reddit_comments_domain_names)), " distinct domain names from the reddit dataset")
print("There are", str(len(num_git_metadata_domain_names)), "distinct domain names from the git dataset")

There are 16844  distinct domain names from the reddit dataset
There are 3444 distinct domain names from the git dataset


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

**Check "Describe and Gather Data" section in README and above**

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
  * I used postgres because spark kept crashing when I performed the regex query on the entire reddit data set. I then tried to use redshift's [REGEXP_SUBSTR function](https://docs.aws.amazon.com/redshift/latest/dg/REGEXP_SUBSTR.html) with no success as the dialect of regex it supports did not have the features I needed therefore I ended up using postgres with its regexp_matches function.
* Propose how often the data should be updated and why.
  * The reddit dataset could ideally be updated in real time live from reddit.
  * The git repo dataset should be updated with new git repos. This can be done via airflow job once a day
  * New git repos can be added and run once a day via airflow job
  * The aggregations table should be updated when new git repos and commits are added once a day
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
   * I would use [AWS RDS](https://aws.amazon.com/rds/) to get a postgres cluster the appriproate size for my dataset. If postgres runs into scaling issues I would look into using using [Snowflake](https://www.snowflake.com/) data lake / data warehouse. Snowflake also has built in functions like  [PARSE_URL — Snowflake Documentation](https://docs.snowflake.com/en/sql-reference/functions/parse_url.html) which would make the project code more simple.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
   * Since both the reddit and git data are all timestamped
 * The database needed to be accessed by 100+ people.
   * Database Read Replicas can be used to make data highly available and are supported on AWS RDS.